In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras, feature_column
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell


In [2]:
# 配置项
# 这个要放到设置中文之前否则还是小方框
plt.style.use("seaborn")

# 指定默认字体 用来正常显示中文标签
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
InteractiveShell.ast_node_interactivity = "all"

In [3]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 6s 1us/step


In [9]:
def create_model():
    model = keras.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(28*28, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax'),
    ])
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']
                 )
    return model

model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [10]:
# 模型在训练的时候 以Checkpoint方式保存

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个保存模型的回调
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                              save_weights_only=True,
                                              verbose=1)
# 使用回调训练模型
model.fit(train_images, 
          train_labels, 
          epochs=10, 
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]
         )

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.3293 - accuracy: 0.6046
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 563us/sample - loss: 1.2205 - accuracy: 0.6360 - val_loss: 0.7428 - val_accuracy: 0.7770
Epoch 2/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.4687 - accuracy: 0.8580
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 280us/sample - loss: 0.4580 - accuracy: 0.8680 - val_loss: 0.5391 - val_accuracy: 0.8340
Epoch 3/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.3019 - accuracy: 0.9135
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 245us/sample - loss: 0.2925 - accuracy: 0.9180 - val_loss: 0.4989 - val_accuracy: 0.8350
Epoch 4/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.2226 - accuracy: 0.9444
Epoch

In [11]:
!ls {checkpoint_dir}


checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


In [12]:
# 测试未经训练的模型 
model = create_model()
# 评估未经训练的模型
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 122us/sample - loss: 2.3582 - accuracy: 0.1000
Untrained model, accuracy: 10.00%


In [13]:
# 再次权重 个人认为checkpoint是记录数据特征
model_load = model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 84us/sample - loss: 0.3899 - accuracy: 0.8770
Untrained model, accuracy: 87.70%


In [23]:
# 每隔5次训练 保存一下checkpoint
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个回调，每 5 个 epochs 保存模型的权重
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                              verbose=1, 
                                              save_weights_only=True, 
#                                               save_freq=5, # 这个参数好像不管用 使用period是正常的
                                              period=5
                                             )
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, 
          train_labels, 
          validation_data=(test_images, test_labels),
          epochs=50, 
          verbose=0,
          callbacks=[cp_callback]
         )


W0819 19:41:17.738143 4556719552 callbacks.py:859] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [24]:
# tf默认保存最近5个checkpoint
latest = latest = tf.train.latest_checkpoint(checkpoint_dir)
# 重新创建新模型 加载最近保存的checkpoint
model_test = create_model()
model_test.load_weights(latest)
# 重新评估模型
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 57us/sample - loss: 0.4844 - accuracy: 0.8690
Untrained model, accuracy: 86.90%


In [25]:
# 将模型保存为HDF5格式
mode_h5 = create_model()
mode_h5.fit(train_images, train_labels, epochs=5)
mode_h5.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 328us/sample - loss: 1.1567 - accuracy: 0.6560
Epoch 2/5
1000/1000 [==============================] - 0s 126us/sample - loss: 0.4290 - accuracy: 0.8760
Epoch 3/5
1000/1000 [==============================] - 0s 142us/sample - loss: 0.2915 - accuracy: 0.9240
Epoch 4/5
1000/1000 [==============================] - 0s 129us/sample - loss: 0.2182 - accuracy: 0.9460
Epoch 5/5
1000/1000 [==============================] - 0s 119us/sample - loss: 0.1707 - accuracy: 0.9580


In [26]:
# 加载模型 测试
model_load = keras.models.load_model('my_model.h5')
loss, acc = model_load.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

W0819 19:54:42.704794 4556719552 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0819 19:54:42.705509 4556719552 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0819 19:54:42.706367 4556719552 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0819 19:54:42.707279 4556719552 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0819 19:54:42.708432 4556719552 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0819 19:54:42.710721 4556719552 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#loading_mechanics for details.

1000/1000 [==============================] - 0s 111us/sample - loss: 0.4182 - accuracy: 0.8600
Restored model, accuracy: 86.00%
